# MCP Connectors

## The Model Context Protocol

This notebook explores the **Model Context Protocol (MCP)** — a standardized way for AI applications to connect to external data sources and tools. MCP defines a client-server architecture where AI agents (clients) connect to capability providers (servers) through a uniform protocol.

MCP servers expose three types of capabilities:
- **Resources** — Read-only data endpoints (like GET requests)
- **Tools** — Callable functions with side effects (like POST requests)
- **Prompts** — Reusable interaction templates

We'll build MCP servers that expose real-world capabilities (product inventory, weather data, file systems, databases, and APIs), then connect to them using both the raw MCP SDK and LangChain adapters.

```
┌────────────────────────────────────────────────────────┐
│                    AI Application                      │
│  ┌──────────────┐  ┌──────────────┐  ┌──────────────┐  │
│  │  LangChain   │  │   Direct     │  │  LangGraph   │  │
│  │  MCP Adapter │  │  MCP Client  │  │    Agent     │  │
│  └──────┬───────┘  └──────┬───────┘  └──────┬───────┘  │
└─────────┼─────────────────┼─────────────────┼──────────┘
          │                 │                 │
    ┌─────┴─────┐     ┌─────┴─────┐     ┌─────┴─────┐
    │   stdio   │     │   stdio   │     │   HTTP    │
    │ transport │     │ transport │     │ transport │
    └─────┬─────┘     └─────┬─────┘     └─────┬─────┘
          │                 │                 │
   ┌──────┴──────┐    ┌─────┴──────┐   ┌──────┴──────┐
   │  Inventory  │    │ File System│   │   Weather   │
   │   Server    │    │   Server   │   │   Server    │
   │ (tools +    │    │(resources) │   │ (tools +    │
   │  resources) │    │            │   │  auth)      │
   └─────────────┘    └────────────┘   └─────────────┘
```

## Setup and Imports

Make sure you have the required packages installed:
```
uv sync
```

In [9]:
import os
import json
import asyncio
from dotenv import load_dotenv

from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage

# MCP SDK imports
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client

# LangChain MCP adapter imports
from langchain_mcp_adapters.client import MultiServerMCPClient
from langchain_mcp_adapters.tools import load_mcp_tools

# Load environment variables
load_dotenv()

# Verify API key
if not os.getenv("OPENAI_API_KEY"):
    raise ValueError("OPENAI_API_KEY not found in environment variables")

# Initialize the LLM
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Create the directory for MCP server files
os.makedirs("mcp_servers", exist_ok=True)

print("Setup complete!")

Setup complete!


## MCP Server Files

MCP servers run as **separate processes** — the client launches them via stdio transport (stdin/stdout communication). We use `%%writefile` to create each server as a `.py` file in the `mcp_servers/` subdirectory that the MCP client can spawn as a subprocess.

We'll create several servers throughout this notebook:
1. **Inventory Server** — E-commerce product catalog (our primary example)
2. **Weather Server** — Weather forecasts with API token authentication
3. **Filesystem Server** — Local file browsing and search
4. **Database Server** — SQLite-backed customer order management
5. **GitHub API Server** — REST API integration with GitHub's public API

In [10]:
%%writefile mcp_servers/inventory_server.py
"""Product Inventory MCP Server — Electronics retailer catalog and stock management."""
import json
from mcp.server.fastmcp import FastMCP

mcp = FastMCP("Inventory")

# In-memory product database
PRODUCTS = {
    "LAPTOP-001": {"name": "ProBook 15", "category": "laptops", "price": 1299.99, "stock": 23,
                   "description": "15-inch professional laptop with 32GB RAM"},
    "LAPTOP-002": {"name": "UltraLight Air", "category": "laptops", "price": 999.99, "stock": 45,
                   "description": "Ultra-portable 13-inch laptop, 1kg weight"},
    "HEADPH-001": {"name": "StudioPro X", "category": "audio", "price": 349.99, "stock": 67,
                   "description": "Noise-cancelling over-ear headphones"},
    "HEADPH-002": {"name": "BudFit Sport", "category": "audio", "price": 129.99, "stock": 150,
                   "description": "Wireless sport earbuds, waterproof"},
    "MONITOR-001": {"name": "ClearView 4K", "category": "monitors", "price": 599.99, "stock": 12,
                    "description": "27-inch 4K IPS display, USB-C"},
    "MONITOR-002": {"name": "UltraWide 34", "category": "monitors", "price": 799.99, "stock": 8,
                    "description": "34-inch ultrawide curved monitor"},
    "KEYBOARD-001": {"name": "MechType Pro", "category": "peripherals", "price": 179.99, "stock": 89,
                     "description": "Mechanical keyboard with hot-swap switches"},
    "MOUSE-001": {"name": "PrecisionGlide", "category": "peripherals", "price": 79.99, "stock": 200,
                  "description": "Ergonomic wireless mouse, 8000 DPI"},
}


@mcp.tool()
def search_products(query: str, category: str | None = None) -> str:
    """Search the product catalog by keyword and optional category filter."""
    results = []
    query_lower = query.lower()
    for pid, product in PRODUCTS.items():
        if category and product["category"] != category:
            continue
        if query_lower in product["name"].lower() or query_lower in product["description"].lower():
            results.append({"id": pid, **product})
    if not results:
        return json.dumps({"results": [], "message": "No products found"})
    return json.dumps({"results": results})


@mcp.tool()
def check_stock(product_id: str) -> str:
    """Check current inventory stock level for a product by its ID."""
    product = PRODUCTS.get(product_id)
    if not product:
        return json.dumps({"error": f"Product {product_id} not found"})
    return json.dumps({
        "product_id": product_id, "name": product["name"],
        "stock": product["stock"],
        "status": "in_stock" if product["stock"] > 0 else "out_of_stock"
    })


@mcp.tool()
def get_price(product_id: str) -> str:
    """Get the current price for a product by its ID."""
    product = PRODUCTS.get(product_id)
    if not product:
        return json.dumps({"error": f"Product {product_id} not found"})
    return json.dumps({"product_id": product_id, "name": product["name"], "price": product["price"]})


@mcp.resource("inventory://catalog")
def get_catalog() -> str:
    """Get the full product catalog."""
    return json.dumps({"products": {pid: p for pid, p in PRODUCTS.items()}}, indent=2)


@mcp.resource("inventory://product/{product_id}")
def get_product(product_id: str) -> str:
    """Get details for a specific product."""
    product = PRODUCTS.get(product_id)
    if not product:
        return json.dumps({"error": f"Product {product_id} not found"})
    return json.dumps({"id": product_id, **product}, indent=2)


@mcp.resource("inventory://categories")
def get_categories() -> str:
    """Get the list of product categories."""
    categories = list(set(p["category"] for p in PRODUCTS.values()))
    return json.dumps({"categories": sorted(categories)})


if __name__ == "__main__":
    mcp.run()

Writing mcp_servers/inventory_server.py


In [ ]:
%%writefile mcp_servers/weather_server.py
"""Weather MCP Server — Demonstrates authentication via environment variables."""
import os
import json
import random
from mcp.server.fastmcp import FastMCP

mcp = FastMCP("Weather")

# Simulated weather data
CONDITIONS = ["Sunny", "Partly Cloudy", "Cloudy", "Light Rain", "Heavy Rain", "Thunderstorm", "Snow", "Fog"]


def _check_auth() -> str | None:
    """Check if API_TOKEN is set. Returns error message or None."""
    token = os.environ.get("API_TOKEN")
    if not token:
        return "Authentication required: API_TOKEN environment variable not set"
    return None


@mcp.tool()
def get_forecast(city: str, days: int = 3) -> str:
    """Get a weather forecast for a city. Requires API_TOKEN to be set."""
    auth_error = _check_auth()
    if auth_error:
        return json.dumps({"error": auth_error})

    random.seed(hash(city))  # Deterministic per city for demo
    forecast = []
    for day in range(days):
        forecast.append({
            "day": day + 1,
            "condition": random.choice(CONDITIONS),
            "high_f": random.randint(55, 85),
            "low_f": random.randint(35, 60),
            "humidity_pct": random.randint(30, 90),
        })
    return json.dumps({"city": city, "forecast": forecast})


@mcp.tool()
def get_alerts(region: str) -> str:
    """Get active weather alerts for a region. Requires API_TOKEN to be set."""
    auth_error = _check_auth()
    if auth_error:
        return json.dumps({"error": auth_error})

    # Simulated alerts
    alerts = [
        {"type": "Wind Advisory", "severity": "moderate",
         "message": f"Gusty winds expected in {region} through tonight."},
    ]
    return json.dumps({"region": region, "alerts": alerts})


if __name__ == "__main__":
    mcp.run()

In [ ]:
%%writefile mcp_servers/filesystem_server.py
"""Filesystem MCP Server — Browse and read local files safely."""
import os
import sys
import json
import glob as globmod
from mcp.server.fastmcp import FastMCP

# Root directory is passed as a command-line argument
ROOT_DIR = os.path.abspath(sys.argv[1]) if len(sys.argv) > 1 else os.getcwd()

mcp = FastMCP("Filesystem")


def _safe_path(path: str) -> str | None:
    """Resolve path and ensure it's within ROOT_DIR. Returns None if unsafe."""
    resolved = os.path.normpath(os.path.join(ROOT_DIR, path))
    if not resolved.startswith(ROOT_DIR):
        return None
    return resolved


@mcp.tool()
def list_directory(path: str = ".") -> str:
    """List files and subdirectories at the given path (relative to root)."""
    safe = _safe_path(path)
    if not safe:
        return json.dumps({"error": "Path outside allowed directory"})
    if not os.path.isdir(safe):
        return json.dumps({"error": f"Not a directory: {path}"})
    entries = []
    for name in sorted(os.listdir(safe)):
        full = os.path.join(safe, name)
        entries.append({"name": name, "type": "dir" if os.path.isdir(full) else "file",
                        "size": os.path.getsize(full) if os.path.isfile(full) else None})
    return json.dumps({"path": path, "entries": entries})


@mcp.tool()
def read_file(path: str) -> str:
    """Read the contents of a file (relative to root). Limited to 10KB."""
    safe = _safe_path(path)
    if not safe:
        return json.dumps({"error": "Path outside allowed directory"})
    if not os.path.isfile(safe):
        return json.dumps({"error": f"Not a file: {path}"})
    try:
        with open(safe, "r", encoding="utf-8", errors="replace") as f:
            content = f.read(10240)  # 10KB limit
        return json.dumps({"path": path, "content": content, "truncated": os.path.getsize(safe) > 10240})
    except Exception as e:
        return json.dumps({"error": str(e)})


@mcp.tool()
def search_files(directory: str, pattern: str) -> str:
    """Search for files matching a glob pattern within a directory."""
    safe = _safe_path(directory)
    if not safe:
        return json.dumps({"error": "Path outside allowed directory"})
    full_pattern = os.path.join(safe, "**", pattern)
    matches = globmod.glob(full_pattern, recursive=True)
    # Convert to relative paths
    relative = [os.path.relpath(m, ROOT_DIR) for m in matches
                if m.startswith(ROOT_DIR)]
    return json.dumps({"pattern": pattern, "directory": directory, "matches": sorted(relative)[:50]})


@mcp.resource("file://summary")
def get_summary() -> str:
    """Get a summary of the root directory."""
    file_count = 0
    dir_count = 0
    for _, dirs, files in os.walk(ROOT_DIR):
        file_count += len(files)
        dir_count += len(dirs)
    return json.dumps({"root": ROOT_DIR, "total_files": file_count, "total_directories": dir_count})


if __name__ == "__main__":
    mcp.run()

In [ ]:
%%writefile mcp_servers/database_server.py
"""Database MCP Server — SQLite-backed customer order management."""
import json
import sqlite3
from contextlib import asynccontextmanager
from collections.abc import AsyncIterator
from dataclasses import dataclass
from mcp.server.fastmcp import FastMCP


@dataclass
class DbContext:
    conn: sqlite3.Connection


@asynccontextmanager
async def db_lifespan(server: FastMCP) -> AsyncIterator[DbContext]:
    """Manage database connection lifecycle."""
    conn = sqlite3.connect(":memory:")
    conn.row_factory = sqlite3.Row

    # Create tables and seed data
    conn.executescript("""
        CREATE TABLE customers (
            id INTEGER PRIMARY KEY, name TEXT, email TEXT
        );
        CREATE TABLE orders (
            id TEXT PRIMARY KEY, customer_id INTEGER, product TEXT,
            quantity INTEGER, total REAL, status TEXT,
            FOREIGN KEY (customer_id) REFERENCES customers(id)
        );
        INSERT INTO customers VALUES (1, 'Alice Johnson', 'alice@example.com');
        INSERT INTO customers VALUES (2, 'Bob Smith', 'bob@example.com');
        INSERT INTO customers VALUES (3, 'Carol Williams', 'carol@example.com');
        INSERT INTO orders VALUES ('ORD-001', 1, 'ProBook 15', 1, 1299.99, 'shipped');
        INSERT INTO orders VALUES ('ORD-002', 1, 'MechType Pro', 2, 359.98, 'delivered');
        INSERT INTO orders VALUES ('ORD-003', 2, 'UltraWide 34', 1, 799.99, 'processing');
        INSERT INTO orders VALUES ('ORD-004', 3, 'BudFit Sport', 3, 389.97, 'shipped');
        INSERT INTO orders VALUES ('ORD-005', 2, 'PrecisionGlide', 1, 79.99, 'delivered');
    """)

    try:
        yield DbContext(conn=conn)
    finally:
        conn.close()


mcp = FastMCP("Database", lifespan=db_lifespan)


@mcp.tool()
def query_orders(customer_name: str) -> str:
    """Look up all orders for a customer by name."""
    ctx: DbContext = mcp.get_context()
    cursor = ctx.conn.execute(
        "SELECT o.id, o.product, o.quantity, o.total, o.status "
        "FROM orders o JOIN customers c ON o.customer_id = c.id "
        "WHERE c.name LIKE ?", (f"%{customer_name}%",)
    )
    rows = [dict(row) for row in cursor.fetchall()]
    return json.dumps({"customer": customer_name, "orders": rows})


@mcp.tool()
def get_order_status(order_id: str) -> str:
    """Check the status of a specific order by order ID."""
    ctx: DbContext = mcp.get_context()
    cursor = ctx.conn.execute(
        "SELECT o.id, c.name as customer, o.product, o.status "
        "FROM orders o JOIN customers c ON o.customer_id = c.id "
        "WHERE o.id = ?", (order_id,)
    )
    row = cursor.fetchone()
    if not row:
        return json.dumps({"error": f"Order {order_id} not found"})
    return json.dumps(dict(row))


@mcp.tool()
def get_sales_summary() -> str:
    """Get an aggregate sales report across all orders."""
    ctx: DbContext = mcp.get_context()
    cursor = ctx.conn.execute(
        "SELECT COUNT(*) as total_orders, SUM(total) as total_revenue, "
        "COUNT(DISTINCT customer_id) as unique_customers "
        "FROM orders"
    )
    row = dict(cursor.fetchone())

    # Get orders by status
    status_cursor = ctx.conn.execute(
        "SELECT status, COUNT(*) as count FROM orders GROUP BY status"
    )
    row["by_status"] = {r["status"]: r["count"] for r in status_cursor.fetchall()}
    return json.dumps(row)


@mcp.resource("db://schema")
def get_schema() -> str:
    """Get the database schema description."""
    return json.dumps({
        "tables": {
            "customers": {"columns": ["id (INTEGER PK)", "name (TEXT)", "email (TEXT)"]},
            "orders": {"columns": ["id (TEXT PK)", "customer_id (FK)", "product (TEXT)",
                                    "quantity (INTEGER)", "total (REAL)", "status (TEXT)"]},
        },
        "sample_statuses": ["processing", "shipped", "delivered"],
    }, indent=2)


if __name__ == "__main__":
    mcp.run()

In [ ]:
%%writefile mcp_servers/github_api_server.py
"""GitHub API MCP Server — Wraps GitHub's public REST API as MCP tools."""
import json
import urllib.request
import urllib.error
from mcp.server.fastmcp import FastMCP

mcp = FastMCP("GitHubAPI")


def _github_get(url: str) -> dict | list:
    """Make a GET request to the GitHub API."""
    req = urllib.request.Request(url, headers={
        "Accept": "application/vnd.github.v3+json",
        "User-Agent": "MCP-GitHub-Server/1.0",
    })
    try:
        with urllib.request.urlopen(req, timeout=10) as resp:
            return json.loads(resp.read().decode())
    except urllib.error.HTTPError as e:
        error_body = e.read().decode() if e.fp else ""
        return {"error": f"GitHub API returned {e.code}", "message": error_body[:200]}
    except Exception as e:
        return {"error": str(e)}


@mcp.tool()
def get_repo_info(owner: str, repo: str) -> str:
    """Get metadata about a GitHub repository (stars, language, description, etc.)."""
    data = _github_get(f"https://api.github.com/repos/{owner}/{repo}")
    if "error" in data:
        return json.dumps(data)
    return json.dumps({
        "full_name": data.get("full_name"),
        "description": data.get("description"),
        "stargazers_count": data.get("stargazers_count"),
        "language": data.get("language"),
        "open_issues_count": data.get("open_issues_count"),
        "forks_count": data.get("forks_count"),
        "created_at": data.get("created_at"),
        "updated_at": data.get("updated_at"),
    })


@mcp.tool()
def list_issues(owner: str, repo: str, state: str = "open") -> str:
    """List issues for a GitHub repository. State can be 'open', 'closed', or 'all'."""
    data = _github_get(f"https://api.github.com/repos/{owner}/{repo}/issues?state={state}&per_page=5")
    if isinstance(data, dict) and "error" in data:
        return json.dumps(data)
    issues = [{
        "number": i.get("number"),
        "title": i.get("title"),
        "state": i.get("state"),
        "created_at": i.get("created_at"),
        "labels": [l["name"] for l in i.get("labels", [])],
    } for i in data[:5]]
    return json.dumps(issues)


@mcp.tool()
def search_repos(query: str) -> str:
    """Search GitHub repositories by keyword. Returns top 5 results."""
    encoded = urllib.parse.quote(query)
    data = _github_get(f"https://api.github.com/search/repositories?q={encoded}&per_page=5")
    if "error" in data:
        return json.dumps(data)
    items = [{
        "full_name": r.get("full_name"),
        "description": (r.get("description") or "")[:100],
        "stargazers_count": r.get("stargazers_count"),
        "language": r.get("language"),
    } for r in data.get("items", [])[:5]]
    return json.dumps({"items": items, "total_count": data.get("total_count", 0)})


if __name__ == "__main__":
    import urllib.parse
    mcp.run()

## Client-Side MCP Usage

This section covers connecting to MCP servers from client code. We'll start with the raw MCP SDK for low-level control, then move to LangChain adapters for agent integration.

### Connecting to MCP Servers

The fundamental client connection pattern has three layers:
1. **`StdioServerParameters`** — Defines the server command and arguments
2. **`stdio_client()`** — Creates the transport (spawns the subprocess)
3. **`ClientSession`** — Provides the API to list and call tools/resources

These are nested async context managers. When the outer context exits, the server subprocess is automatically terminated.

In [ ]:
# Connect to the inventory server using the raw MCP SDK
server_params = StdioServerParameters(
    command="python",
    args=["mcp_servers/inventory_server.py"],
)

async with stdio_client(server_params) as (read, write):
    async with ClientSession(read, write) as session:
        await session.initialize()

        # Discover available tools
        tools_result = await session.list_tools()
        print("Available Tools:")
        for tool in tools_result.tools:
            print(f"  - {tool.name}: {tool.description}")

        # Discover available resources
        resources_result = await session.list_resources()
        print("\nAvailable Resources:")
        for resource in resources_result.resources:
            print(f"  - {resource.uri}: {resource.name}")

        # Call a tool
        print("\n--- Calling search_products('laptop') ---")
        result = await session.call_tool("search_products", {"query": "laptop"})
        print(json.dumps(json.loads(result.content[0].text), indent=2))

### LangChain MCP Adapters

`MultiServerMCPClient` from `langchain-mcp-adapters` bridges MCP tools into LangChain's tool ecosystem. It can connect to multiple MCP servers at once, loading all their tools as `BaseTool` instances that work with any LangChain agent.

The key value: you get MCP's standardized server interface **plus** LangChain's agent reasoning — the agent decides which tools to call and when.

In [ ]:
from langchain.agents import create_agent

# Connect to the inventory server via LangChain adapter
async with MultiServerMCPClient(
    {
        "inventory": {
            "command": "python",
            "args": ["mcp_servers/inventory_server.py"],
            "transport": "stdio",
        }
    }
) as client:
    # Load MCP tools as LangChain tools
    tools = client.get_tools()

    print("LangChain Tools loaded from MCP:")
    for tool in tools:
        print(f"  - {tool.name}: {tool.description[:80]}...")

    # Create an agent that uses MCP tools
    agent = create_agent(
        model="gpt-4o-mini",
        tools=tools,
        prompt="You are a helpful shopping assistant for an electronics store. "
               "Use the available tools to help customers find products and check availability.",
    )

    response = await agent.ainvoke(
        {"messages": [{"role": "user", "content": "I need a lightweight laptop for travel. What do you have?"}]}
    )

    # Display the agent's final response
    for msg in response["messages"]:
        if hasattr(msg, "type") and msg.type == "ai" and msg.content and not getattr(msg, "tool_calls", []):
            print(f"\nAgent Response:\n{msg.content}")

### Using MCP for Enhanced Retrieval

MCP resources and tools complement each other for retrieval:
- **Resources** provide bulk data (like reading an entire catalog) — useful for broad context
- **Tools** perform targeted queries (like searching for specific products) — useful for precision

Combining both in a pipeline gives you a RAG-like pattern: first scan the broad context, then drill into specifics, then hand everything to the LLM for a grounded response.

In [ ]:
# Enhanced retrieval: combine resources (bulk data) with tools (targeted queries)
server_params = StdioServerParameters(command="python", args=["mcp_servers/inventory_server.py"])

async with stdio_client(server_params) as (read, write):
    async with ClientSession(read, write) as session:
        await session.initialize()

        # Step 1: Read the categories resource for catalog structure
        print("Step 1: Read catalog structure via resource")
        categories = await session.read_resource("inventory://categories")
        cats = json.loads(categories.contents[0].text)
        print(f"  Categories: {cats['categories']}")

        # Step 2: Read full catalog resource for broad context
        print("\nStep 2: Read full catalog via resource")
        catalog = await session.read_resource("inventory://catalog")
        catalog_data = json.loads(catalog.contents[0].text)
        print(f"  Total products: {len(catalog_data['products'])}")

        # Step 3: Use tool for targeted search
        print("\nStep 3: Targeted search via tool")
        result = await session.call_tool("search_products", {"query": "wireless", "category": "audio"})
        search_results = json.loads(result.content[0].text)
        print(f"  Found {len(search_results['results'])} wireless audio products")
        for p in search_results["results"]:
            print(f"    - {p['name']}: ${p['price']}")

        # Step 4: Feed combined context to LLM for a grounded recommendation
        print("\nStep 4: LLM recommendation grounded in MCP data")
        context = f"Catalog has {len(catalog_data['products'])} products across categories: {cats['categories']}.\n"
        context += f"Wireless audio options: {json.dumps(search_results['results'], indent=2)}"

        recommendation = llm.invoke([
            SystemMessage(content="You are a product advisor. Recommend a product based on the catalog data provided. Be concise."),
            HumanMessage(content=f"Customer wants wireless audio for running.\n\n{context}")
        ])
        print(f"  Recommendation: {recommendation.content}")

### Authentication and Authorization

MCP supports two authentication patterns:

1. **Environment variables (stdio transport)** — Pass credentials via the `env` parameter of `StdioServerParameters`. The server checks `os.environ` on its side.
2. **HTTP headers (HTTP/SSE transport)** — Pass `Authorization` headers in the `MultiServerMCPClient` configuration for remote servers.

Our weather server requires an `API_TOKEN` environment variable, demonstrating the stdio pattern.

In [ ]:
import sys

# Pattern 1: Auth via environment variables (stdio transport)

# First, connect WITHOUT the token
print("--- Connecting WITHOUT auth token ---")
server_params_no_auth = StdioServerParameters(
    command=sys.executable,
    args=["mcp_servers/weather_server.py"],
    env={"PATH": os.environ.get("PATH", "")},  # No API_TOKEN
)

async with stdio_client(server_params_no_auth) as (read, write):
    async with ClientSession(read, write) as session:
        await session.initialize()
        result = await session.call_tool("get_forecast", {"city": "San Francisco"})
        data = json.loads(result.content[0].text)
        print(f"  Result: {data}")
        # Should show an auth error

# Now connect WITH the token
print("\n--- Connecting WITH auth token ---")
server_params_auth = StdioServerParameters(
    command=sys.executable,
    args=["mcp_servers/weather_server.py"],
    env={
        "PATH": os.environ.get("PATH", ""),
        "API_TOKEN": "demo-weather-token-123",
    },
)

async with stdio_client(server_params_auth) as (read, write):
    async with ClientSession(read, write) as session:
        await session.initialize()
        result = await session.call_tool("get_forecast", {"city": "San Francisco"})
        data = json.loads(result.content[0].text)
        print(f"  Forecast: {json.dumps(data, indent=2)}")

# Pattern 2: Auth via HTTP headers (for remote servers)
print("\n--- HTTP transport auth pattern (configuration only) ---")
http_config = {
    "weather_remote": {
        "url": "https://api.example.com/mcp",
        "transport": "streamable_http",
        "headers": {
            "Authorization": "Bearer YOUR_API_KEY_HERE",
            "X-Client-Id": "notebook-demo",
        },
    }
}
print(f"  Config: {json.dumps(http_config, indent=2)}")
print("  (Not connecting — this shows the configuration pattern for remote servers)")

## Building MCP Connectors

This section covers building MCP servers from scratch — lifecycle management, exposing resources and tools, and handling errors robustly.

### Connector Structure and Lifecycle

A `FastMCP` server follows a clear lifecycle:
1. **Creation** — `mcp = FastMCP("Name")` with optional `lifespan` parameter
2. **Registration** — `@mcp.tool()`, `@mcp.resource()`, `@mcp.prompt()` decorators register capabilities
3. **Startup** — The `lifespan` async context manager runs, initializing shared state (database connections, caches, etc.)
4. **Running** — Server processes requests; shared state is accessible via `mcp.get_context()`
5. **Shutdown** — The `lifespan` context manager cleans up resources

Let's review the inventory server code to see this structure in practice, then connect to it as a client to observe the lifecycle:

In [ ]:
# Review the inventory server structure (written to disk in the Server Files section)
with open("mcp_servers/inventory_server.py") as f:
    print(f.read())

In [ ]:
# Connect to the inventory server and observe the lifecycle
server_params = StdioServerParameters(command="python", args=["mcp_servers/inventory_server.py"])

async with stdio_client(server_params) as (read, write):
    async with ClientSession(read, write) as session:
        await session.initialize()
        print("Connected to inventory server\n")

        # Discover tools registered via @mcp.tool()
        tools = await session.list_tools()
        print(f"Tools ({len(tools.tools)}):")
        for tool in tools.tools:
            print(f"  - {tool.name}: {tool.description}")

        # Discover resources registered via @mcp.resource()
        resources = await session.list_resources()
        print(f"\nResources ({len(resources.resources)}):")
        for r in resources.resources:
            print(f"  - {r.uri}")

        # Use tools — state is managed by the server process
        result = await session.call_tool("search_products", {"query": "laptop"})
        print(f"\nSearch result: {result.content[0].text}")

        result = await session.call_tool("check_stock", {"product_id": "LAPTOP-001"})
        print(f"Stock check: {result.content[0].text}")

        # Read a resource
        result = await session.read_resource("inventory://categories")
        print(f"Categories: {result.contents[0].text}")

# Exiting the context triggers shutdown (lifespan cleanup if configured)
print("\nServer shut down — session and subprocess terminated")

### Implementing Resource Access

MCP resources expose read-only data through URI patterns:
- **Static resources** have fixed URIs (e.g., `inventory://catalog`) — they appear in `list_resources()`
- **Resource templates** have parameterized URIs (e.g., `inventory://product/{product_id}`) — they appear in `list_resource_templates()`

Resources are defined with `@mcp.resource("uri://pattern")` and return string data.

In [ ]:
# Demonstrate resource access patterns using the inventory server
server_params = StdioServerParameters(command="python", args=["mcp_servers/inventory_server.py"])

async with stdio_client(server_params) as (read, write):
    async with ClientSession(read, write) as session:
        await session.initialize()

        # List static resources
        resources = await session.list_resources()
        print("Static Resources:")
        for r in resources.resources:
            print(f"  {r.uri} — {r.name}")

        # List resource templates (parameterized)
        templates = await session.list_resource_templates()
        print("\nResource Templates:")
        for t in templates.resource_templates:
            print(f"  {t.uri_template} — {t.name}")

        # Read a static resource
        print("\n--- Reading inventory://categories ---")
        result = await session.read_resource("inventory://categories")
        print(json.dumps(json.loads(result.contents[0].text), indent=2))

        # Read a templated resource — product details
        print("\n--- Reading inventory://product/LAPTOP-001 ---")
        result = await session.read_resource("inventory://product/LAPTOP-001")
        print(json.dumps(json.loads(result.contents[0].text), indent=2))

        print("\n--- Reading inventory://product/HEADPH-001 ---")
        result = await session.read_resource("inventory://product/HEADPH-001")
        print(json.dumps(json.loads(result.contents[0].text), indent=2))

### Implementing Tool Exposure

The `@mcp.tool()` decorator auto-generates a JSON Schema from the function's:
- **Name** — the function name becomes the tool name
- **Docstring** — becomes the tool's description (shown to LLMs)
- **Type hints** — become the input schema (parameters, types, required/optional)

Good tool design means clear names, typed parameters, and descriptive docstrings — because the LLM uses all of this to decide *when* and *how* to call each tool.

In [ ]:
# Inspect tool schemas and exercise each tool
server_params = StdioServerParameters(command="python", args=["mcp_servers/inventory_server.py"])

async with stdio_client(server_params) as (read, write):
    async with ClientSession(read, write) as session:
        await session.initialize()

        # Inspect the auto-generated schemas
        tools = await session.list_tools()
        print("Tool Schemas (auto-generated from type hints):")
        for tool in tools.tools:
            print(f"\n  {tool.name}:")
            print(f"    Description: {tool.description}")
            print(f"    Input Schema: {json.dumps(tool.inputSchema, indent=6)}")

        # Exercise: search with just a query
        print("\n--- search_products(query='monitor') ---")
        r = await session.call_tool("search_products", {"query": "monitor"})
        print(json.dumps(json.loads(r.content[0].text), indent=2))

        # Exercise: search with query + category filter
        print("\n--- search_products(query='wireless', category='audio') ---")
        r = await session.call_tool("search_products", {"query": "wireless", "category": "audio"})
        print(json.dumps(json.loads(r.content[0].text), indent=2))

        # Exercise: check_stock
        print("\n--- check_stock(product_id='MONITOR-002') ---")
        r = await session.call_tool("check_stock", {"product_id": "MONITOR-002"})
        print(json.dumps(json.loads(r.content[0].text), indent=2))

        # Exercise: get_price
        print("\n--- get_price(product_id='KEYBOARD-001') ---")
        r = await session.call_tool("get_price", {"product_id": "KEYBOARD-001"})
        print(json.dumps(json.loads(r.content[0].text), indent=2))

### Error Handling and Retries

MCP tool calls can fail in several ways:
- **Server-side validation errors** — The tool runs but returns an error message (e.g., "product not found")
- **Tool not found** — The client requests a tool that doesn't exist
- **Transport errors** — The server process crashes or times out

A retry wrapper with exponential backoff handles transient failures gracefully. The `result.isError` flag indicates whether the tool returned an error response.

In [ ]:
async def mcp_call_with_retry(session, tool_name, arguments, max_retries=3, base_delay=0.5):
    """Call an MCP tool with exponential backoff retry logic."""
    for attempt in range(max_retries):
        try:
            result = await session.call_tool(tool_name, arguments)
            # Check for MCP-level errors
            if result.isError:
                print(f"  [Attempt {attempt + 1}] Tool returned error: {result.content[0].text}")
                if attempt < max_retries - 1:
                    delay = base_delay * (2 ** attempt)
                    print(f"  Retrying in {delay}s...")
                    await asyncio.sleep(delay)
                    continue
            return result
        except Exception as e:
            print(f"  [Attempt {attempt + 1}] Exception: {type(e).__name__}: {e}")
            if attempt < max_retries - 1:
                delay = base_delay * (2 ** attempt)
                print(f"  Retrying in {delay}s...")
                await asyncio.sleep(delay)
            else:
                raise
    return None


# Test error handling scenarios
server_params = StdioServerParameters(command="python", args=["mcp_servers/inventory_server.py"])

async with stdio_client(server_params) as (read, write):
    async with ClientSession(read, write) as session:
        await session.initialize()

        # Case 1: Valid call (baseline)
        print("--- Case 1: Valid call ---")
        result = await mcp_call_with_retry(session, "check_stock", {"product_id": "LAPTOP-001"})
        print(f"  Result: {result.content[0].text}")

        # Case 2: Invalid product ID (server returns error JSON, not a protocol error)
        print("\n--- Case 2: Invalid product ID ---")
        result = await mcp_call_with_retry(session, "check_stock", {"product_id": "NONEXISTENT-999"})
        print(f"  Result: {result.content[0].text}")

        # Case 3: Nonexistent tool (protocol-level error)
        print("\n--- Case 3: Nonexistent tool ---")
        try:
            result = await mcp_call_with_retry(session, "delete_everything", {}, max_retries=1)
        except Exception as e:
            print(f"  Caught expected error: {type(e).__name__}: {e}")

        print("\nError handling demonstration complete!")

## Integration Patterns

This section covers common integration patterns — connecting MCP to file systems, databases, external APIs, creating composite tools, and composing multiple servers into a single agent.

### File System Access

A file system MCP server lets an AI agent browse and read local files. This is useful for document processing, code analysis, or any workflow that needs access to local data.

**Security is critical**: the server validates all paths to prevent directory traversal attacks (`../../../etc/passwd`). Every path is resolved and checked against the allowed root directory.

In [ ]:
# Connect to the filesystem server, pointed at the documents directory
docs_path = os.path.abspath("documents")
if not os.path.isdir(docs_path):
    print(f"Warning: '{docs_path}' not found. Using current directory instead.")
    docs_path = os.getcwd()

server_params = StdioServerParameters(
    command="python",
    args=["mcp_servers/filesystem_server.py", docs_path],
)

async with stdio_client(server_params) as (read, write):
    async with ClientSession(read, write) as session:
        await session.initialize()

        # List the root directory
        print("--- List root directory ---")
        result = await session.call_tool("list_directory", {"path": "."})
        entries = json.loads(result.content[0].text)
        for entry in entries.get("entries", [])[:10]:
            icon = "📁" if entry["type"] == "dir" else "📄"
            size = f" ({entry['size']} bytes)" if entry.get("size") else ""
            print(f"  {icon} {entry['name']}{size}")
        if len(entries.get("entries", [])) > 10:
            print(f"  ... and {len(entries['entries']) - 10} more")

        # Search for markdown files
        print("\n--- Search for .md files ---")
        result = await session.call_tool("search_files", {"directory": ".", "pattern": "*.md"})
        files = json.loads(result.content[0].text)
        for f in files.get("matches", [])[:5]:
            print(f"  {f}")

        # Read a file
        if files.get("matches"):
            first_file = files["matches"][0]
            print(f"\n--- Reading {first_file} ---")
            result = await session.call_tool("read_file", {"path": first_file})
            data = json.loads(result.content[0].text)
            content = data.get("content", "")
            print(content[:500] + "..." if len(content) > 500 else content)

        # Read directory summary resource
        print("\n--- Directory summary ---")
        summary = await session.read_resource("file://summary")
        print(summary.contents[0].text)

### Database Connections

The database server demonstrates a production-ready pattern:
- The **lifespan** function manages the database connection (create on startup, close on shutdown)
- **Tools** expose safe query operations — no raw SQL from the client
- **Resources** expose schema metadata so clients understand the data model

Here we use SQLite, but the same pattern works for PostgreSQL, MySQL, or any database.

In [ ]:
# Connect to the database server
server_params = StdioServerParameters(command="python", args=["mcp_servers/database_server.py"])

async with stdio_client(server_params) as (read, write):
    async with ClientSession(read, write) as session:
        await session.initialize()

        # Read the database schema
        print("--- Database Schema ---")
        schema = await session.read_resource("db://schema")
        print(schema.contents[0].text)

        # Query orders for a customer
        print("\n--- Orders for 'Alice Johnson' ---")
        result = await session.call_tool("query_orders", {"customer_name": "Alice Johnson"})
        print(json.dumps(json.loads(result.content[0].text), indent=2))

        # Check a specific order status
        print("\n--- Order status for ORD-003 ---")
        result = await session.call_tool("get_order_status", {"order_id": "ORD-003"})
        print(json.dumps(json.loads(result.content[0].text), indent=2))

        # Get the sales summary
        print("\n--- Sales Summary ---")
        result = await session.call_tool("get_sales_summary", {})
        print(json.dumps(json.loads(result.content[0].text), indent=2))

### API Integrations (REST, GraphQL)

Wrapping an external REST API as an MCP server creates a clean abstraction: the AI agent calls MCP tools, and the server handles the HTTP requests, authentication, rate limiting, and error handling.

Our GitHub API server wraps GitHub's public REST API using only `urllib` (no extra dependencies). It demonstrates key considerations for API integrations: timeout handling, HTTP error codes, and response shaping.

In [ ]:
# Connect to the GitHub API server (makes live API calls)
server_params = StdioServerParameters(command="python", args=["mcp_servers/github_api_server.py"])

async with stdio_client(server_params) as (read, write):
    async with ClientSession(read, write) as session:
        await session.initialize()

        # Search for repos
        print("--- Search repos: 'langchain python' ---")
        result = await session.call_tool("search_repos", {"query": "langchain python"})
        repos = json.loads(result.content[0].text)
        for repo in repos.get("items", [])[:3]:
            print(f"  {repo['full_name']} ({repo['stargazers_count']} stars) — {repo.get('language', 'N/A')}")

        # Get repo info
        print("\n--- Repo info: langchain-ai/langchain ---")
        result = await session.call_tool("get_repo_info", {"owner": "langchain-ai", "repo": "langchain"})
        info = json.loads(result.content[0].text)
        print(f"  Stars: {info.get('stargazers_count')}")
        print(f"  Language: {info.get('language')}")
        print(f"  Forks: {info.get('forks_count')}")
        print(f"  Description: {(info.get('description') or '')[:100]}")

        # List recent issues
        print("\n--- Recent open issues: modelcontextprotocol/python-sdk ---")
        result = await session.call_tool(
            "list_issues", {"owner": "modelcontextprotocol", "repo": "python-sdk", "state": "open"}
        )
        issues = json.loads(result.content[0].text)
        if isinstance(issues, list):
            for issue in issues[:3]:
                labels = ", ".join(issue.get("labels", [])) or "no labels"
                print(f"  #{issue['number']}: {issue['title'][:60]} [{labels}]")
        else:
            print(f"  {issues}")

### Custom Tool Creation

MCP tools become especially powerful when composed by an LLM agent. Instead of a human deciding which tools to call in what order, the agent reasons about the user's intent and orchestrates multiple tool calls automatically.

Here we load the inventory server's tools into a LangChain agent and give it a *comparison* task — the agent must call multiple tools and synthesize the results.

In [ ]:
from langchain.agents import create_agent

# Load inventory tools via LangChain adapter
async with MultiServerMCPClient(
    {
        "inventory": {
            "command": "python",
            "args": ["mcp_servers/inventory_server.py"],
            "transport": "stdio",
        }
    }
) as client:
    tools = client.get_tools()

    # Create an agent specialized for product comparison
    agent = create_agent(
        model="gpt-4o-mini",
        tools=tools,
        prompt=(
            "You are a product comparison assistant. When a customer asks to compare products:\n"
            "1. Search for relevant products\n"
            "2. Get prices for each\n"
            "3. Check stock availability\n"
            "4. Provide a structured comparison with a clear recommendation"
        ),
    )

    response = await agent.ainvoke(
        {"messages": [{"role": "user", "content": "Compare your two laptop options. Which is better for someone on a budget?"}]}
    )

    # Show tool calls and final response
    for msg in response["messages"]:
        if hasattr(msg, "tool_calls") and msg.tool_calls:
            for tc in msg.tool_calls:
                print(f"[Tool Call] {tc['name']}({tc['args']})")
        if hasattr(msg, "type") and msg.type == "ai" and msg.content and not getattr(msg, "tool_calls", []):
            print(f"\nComparison:\n{msg.content}")

### Composing Multiple MCP Servers

`MultiServerMCPClient` can connect to **any number** of MCP servers simultaneously, loading all their tools into a single pool. The LangChain agent then picks the right tools for each query.

This is MCP's key value proposition: **uniform access to diverse data sources**. Add a new data source by configuring one more MCP server — no changes to the agent code.

In [ ]:
from langchain.agents import create_agent

# Connect to multiple MCP servers simultaneously
async with MultiServerMCPClient(
    {
        "inventory": {
            "command": "python",
            "args": ["mcp_servers/inventory_server.py"],
            "transport": "stdio",
        },
        "github": {
            "command": "python",
            "args": ["mcp_servers/github_api_server.py"],
            "transport": "stdio",
        },
    }
) as multi_client:
    all_tools = multi_client.get_tools()

    print(f"Total tools loaded from all servers: {len(all_tools)}")
    for tool in all_tools:
        print(f"  - {tool.name}: {tool.description[:60]}...")

    # Create a multi-source agent
    agent = create_agent(
        model="gpt-4o-mini",
        tools=all_tools,
        prompt=(
            "You are a helpful assistant with access to a product inventory system "
            "and GitHub. Use the appropriate tools based on the user's question."
        ),
    )

    # Test 1: Product question (should use inventory tools)
    print("\n" + "=" * 50)
    print("Question about products:")
    print("=" * 50)
    r1 = await agent.ainvoke(
        {"messages": [{"role": "user", "content": "What monitors do you have in stock and how much do they cost?"}]}
    )
    for msg in r1["messages"]:
        if hasattr(msg, "type") and msg.type == "ai" and msg.content and not getattr(msg, "tool_calls", []):
            print(msg.content)

    # Test 2: GitHub question (should use GitHub tools)
    print("\n" + "=" * 50)
    print("Question about GitHub:")
    print("=" * 50)
    r2 = await agent.ainvoke(
        {"messages": [{"role": "user", "content": "How many stars does the modelcontextprotocol/python-sdk repo have?"}]}
    )
    for msg in r2["messages"]:
        if hasattr(msg, "type") and msg.type == "ai" and msg.content and not getattr(msg, "tool_calls", []):
            print(msg.content)

## Testing MCP Connectors

This section covers strategies for testing MCP servers and client integrations — from local protocol-level tests to full end-to-end integration tests to mock servers for development.

### Local Testing Strategies

The most reliable way to test an MCP server is to connect to it as a real client and verify every tool and resource. This tests the full protocol round-trip — serialization, transport, execution, and response parsing.

Each test case follows the pattern: call a tool, parse the result, assert on the expected output.

In [ ]:
async def test_inventory_server():
    """Test suite for the inventory MCP server."""
    passed = 0
    failed = 0

    server_params = StdioServerParameters(command="python", args=["mcp_servers/inventory_server.py"])

    async with stdio_client(server_params) as (read, write):
        async with ClientSession(read, write) as session:
            await session.initialize()

            # Test 1: Tool discovery
            try:
                tools = await session.list_tools()
                tool_names = [t.name for t in tools.tools]
                assert "search_products" in tool_names
                assert "check_stock" in tool_names
                assert "get_price" in tool_names
                passed += 1
                print("[PASS] Tool discovery — found all 3 expected tools")
            except AssertionError as e:
                failed += 1
                print(f"[FAIL] Tool discovery: {e}")

            # Test 2: Search returns results for known products
            try:
                r = await session.call_tool("search_products", {"query": "laptop"})
                data = json.loads(r.content[0].text)
                assert len(data["results"]) >= 1, "Expected at least one laptop result"
                assert any("laptop" in p["name"].lower() or "laptop" in p["description"].lower()
                           for p in data["results"]), "Results should mention 'laptop'"
                passed += 1
                print(f"[PASS] Search products — returned {len(data['results'])} laptop(s)")
            except (AssertionError, Exception) as e:
                failed += 1
                print(f"[FAIL] Search products: {e}")

            # Test 3: Stock data is valid
            try:
                r = await session.call_tool("check_stock", {"product_id": "LAPTOP-001"})
                data = json.loads(r.content[0].text)
                assert "stock" in data, "Response must contain 'stock' field"
                assert isinstance(data["stock"], int), "Stock must be an integer"
                assert data["stock"] > 0, "LAPTOP-001 should be in stock"
                passed += 1
                print(f"[PASS] Check stock — LAPTOP-001 has {data['stock']} units")
            except (AssertionError, Exception) as e:
                failed += 1
                print(f"[FAIL] Check stock: {e}")

            # Test 4: Resources are readable
            try:
                result = await session.read_resource("inventory://categories")
                cats = json.loads(result.contents[0].text)
                assert "categories" in cats, "Must contain 'categories' key"
                assert len(cats["categories"]) >= 2, "Must have at least 2 categories"
                passed += 1
                print(f"[PASS] Resource access — {len(cats['categories'])} categories")
            except (AssertionError, Exception) as e:
                failed += 1
                print(f"[FAIL] Resource access: {e}")

            # Test 5: Invalid product returns error gracefully
            try:
                r = await session.call_tool("check_stock", {"product_id": "INVALID-999"})
                data = json.loads(r.content[0].text)
                assert "error" in data, "Invalid product should return error"
                passed += 1
                print("[PASS] Error handling — invalid product returns error message")
            except (AssertionError, Exception) as e:
                failed += 1
                print(f"[FAIL] Error handling: {e}")

    print(f"\nResults: {passed} passed, {failed} failed")
    return passed, failed


# Run the tests
await test_inventory_server()

### Integration Testing

Integration testing verifies the full chain: **user question → LangChain agent → MCP tool call → server execution → response**. This catches issues that protocol-level tests miss — like the LLM failing to select the right tool, or the agent misinterpreting tool output.

We verify both that the agent *used* tools and that the final response contains expected information.

In [ ]:
from langchain.agents import create_agent

async def test_agent_integration():
    """Integration test: LLM agent + MCP server end-to-end."""
    print("--- Integration Test: Agent + Inventory Server ---\n")

    async with MultiServerMCPClient(
        {
            "inventory": {
                "command": "python",
                "args": ["mcp_servers/inventory_server.py"],
                "transport": "stdio",
            }
        }
    ) as client:
        tools = client.get_tools()

        agent = create_agent(
            model="gpt-4o-mini",
            tools=tools,
            prompt="You are a store assistant. Use tools to answer product questions accurately.",
        )

        response = await agent.ainvoke(
            {"messages": [{"role": "user", "content": "What headphones do you sell and how much do they cost?"}]}
        )

        # Check: did the agent use tools?
        tool_calls_made = [
            msg for msg in response["messages"]
            if hasattr(msg, "tool_calls") and msg.tool_calls
        ]
        print(f"Tool calls made: {len(tool_calls_made)}")
        for msg in tool_calls_made:
            for tc in msg.tool_calls:
                print(f"  Called: {tc['name']}({tc['args']})")

        # Extract the final answer
        final_answer = ""
        for msg in response["messages"]:
            if hasattr(msg, "type") and msg.type == "ai" and msg.content and not getattr(msg, "tool_calls", []):
                final_answer = msg.content

        print(f"\nAgent response:\n{final_answer[:400]}...\n" if len(final_answer) > 400 else f"\nAgent response:\n{final_answer}\n")

        # Verify the response
        checks = {
            "Agent used tools": len(tool_calls_made) > 0,
            "Mentions headphones": any(w in final_answer.lower() for w in ["headphone", "studiopro", "budfit", "earbud"]),
            "Mentions a price": any(p in final_answer for p in ["$", "349", "129", "price"]),
        }

        print("Integration checks:")
        all_passed = True
        for check_name, check_passed in checks.items():
            status = "PASS" if check_passed else "FAIL"
            print(f"  [{status}] {check_name}")
            if not check_passed:
                all_passed = False

        return all_passed


success = await test_agent_integration()
print(f"\nOverall: {'PASS' if success else 'FAIL'}")

### Mock Servers for Development

When developing a client against an MCP server that calls expensive external APIs (or that hasn't been built yet), create a **mock server** with the same tool interface but hardcoded responses.

This allows rapid client development and testing:
- No network calls (fast, free, offline)
- Predictable responses (easy to assert on)
- Same interface (swap mock for real with a config change)

In [ ]:
%%writefile mcp_servers/mock_github_server.py
"""Mock GitHub API server — returns canned data for development and testing."""
import json
from mcp.server.fastmcp import FastMCP

mcp = FastMCP("MockGitHub")

MOCK_REPO = {
    "full_name": "mock-org/mock-repo",
    "description": "A mock repository for testing",
    "stargazers_count": 1234,
    "language": "Python",
    "open_issues_count": 5,
    "forks_count": 42,
    "created_at": "2024-01-01T00:00:00Z",
    "updated_at": "2025-06-15T12:00:00Z",
}

MOCK_ISSUES = [
    {"number": 1, "title": "Mock issue one", "state": "open", "created_at": "2025-06-01", "labels": ["bug"]},
    {"number": 2, "title": "Mock issue two", "state": "open", "created_at": "2025-06-10", "labels": ["feature"]},
    {"number": 3, "title": "Mock issue three", "state": "closed", "created_at": "2025-05-20", "labels": []},
]

MOCK_SEARCH = [
    {"full_name": "mock-org/result-1", "description": "First search result", "stargazers_count": 500, "language": "Python"},
    {"full_name": "mock-org/result-2", "description": "Second search result", "stargazers_count": 200, "language": "TypeScript"},
]


@mcp.tool()
def get_repo_info(owner: str, repo: str) -> str:
    """Get metadata about a GitHub repository (stars, language, description, etc.)."""
    return json.dumps({**MOCK_REPO, "full_name": f"{owner}/{repo}"})


@mcp.tool()
def list_issues(owner: str, repo: str, state: str = "open") -> str:
    """List issues for a GitHub repository. State can be 'open', 'closed', or 'all'."""
    filtered = [i for i in MOCK_ISSUES if state == "all" or i["state"] == state]
    return json.dumps(filtered)


@mcp.tool()
def search_repos(query: str) -> str:
    """Search GitHub repositories by keyword. Returns top 5 results."""
    return json.dumps({"items": MOCK_SEARCH, "total_count": len(MOCK_SEARCH)})


if __name__ == "__main__":
    mcp.run()

In [ ]:
# Compare the mock and real GitHub servers
async def run_against_server(label: str, server_args: list[str]):
    """Run the same client code against any server with the GitHub tool interface."""
    server_params = StdioServerParameters(command="python", args=server_args)

    async with stdio_client(server_params) as (read, write):
        async with ClientSession(read, write) as session:
            await session.initialize()

            tools = await session.list_tools()
            tool_names = sorted([t.name for t in tools.tools])
            print(f"  Tools: {tool_names}")

            result = await session.call_tool("get_repo_info", {"owner": "langchain-ai", "repo": "langchain"})
            data = json.loads(result.content[0].text)
            print(f"  Repo: {data.get('full_name')} ({data.get('stargazers_count')} stars)")

            result = await session.call_tool("search_repos", {"query": "mcp python"})
            search = json.loads(result.content[0].text)
            print(f"  Search results: {search.get('total_count', len(search.get('items', [])))} repos found")


# Mock server — instant, offline, predictable
print("--- Mock GitHub Server ---")
await run_against_server("mock", ["mcp_servers/mock_github_server.py"])

# Real server — live API calls (may be rate-limited)
print("\n--- Real GitHub Server ---")
await run_against_server("real", ["mcp_servers/github_api_server.py"])

print("\nBoth servers expose identical tool interfaces.")
print("Use mock during development, swap to real for integration testing.")